In [1]:
import random
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from custom_dataset import CustomDataset, make_data, make_roberta_data, RobertaDataset
from model import BaseModel, RobertaDocument, RobertaLinear

from datetime import datetime, timedelta

from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoConfig

from sklearn import preprocessing

from arguments import parse_args
from collections import defaultdict

import argparse
import os
import torch

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

num_cores = os.cpu_count()
args = parse_args()
seed_everything(args.seed)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [2]:
df, test,label_encoder = make_roberta_data(args)
test = test.reset_index(drop =True)

tokenizer = AutoTokenizer.from_pretrained(args.PLM)
config = AutoConfig.from_pretrained(args.PLM)

test_dataset = RobertaDataset(
    test.reset_index(drop=True),
    tokenizer,
    args.max_input_length,
    None
)
test_loader = DataLoader(test_dataset, batch_size = args.batch_size, shuffle=False, num_workers = num_cores)

In [3]:
def inference_ensemble(model_dir, test_loader, device):
    softmax_layer = nn.Softmax(dim=1)
    dirs = os.listdir(model_dir)
    dirs = sorted(dirs)
    print("@@@@@@@@@@@@@@@@@@@@@")
    print(dirs)
    print("@@@@@@@@@@@@@@@@@@@@@")

    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []

    for i in range(len(dirs)):
        tmp_dirs = os.listdir(os.path.join(model_dir,dirs[i]))
        tmp_dirs = sorted(tmp_dirs)

        config = AutoConfig.from_pretrained(args.PLM)
        config.model_name = args.model_name

        print("@@@@@@@@@@@@@@@@")
        print("model_load start")

        print(f"model : {tmp_dirs[1]}")
        print(f"model_state_dict : {tmp_dirs[0]}")
        
        model = torch.load(os.path.join(model_dir,dirs[i],tmp_dirs[1]))
        model.load_state_dict(torch.load(os.path.join(model_dir,dirs[i],tmp_dirs[0])))

        print("model loaded")
        print("@@@@@@@@@@@@@@@@")

        model.to(device)
        model.eval()

        tmp_type_preds, tmp_polarity_preds, tmp_tense_preds, tmp_certainty_preds = [], [], [], []

        with torch.no_grad():
            for input_ids, attention_mask in tqdm(test_loader):
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)  

                type_logit, polarity_logit, tense_logit, certainty_logit = model(input_ids, attention_mask)

                soft_type_logit = softmax_layer(type_logit).detach().cpu()
                soft_polarity_logit = softmax_layer(polarity_logit).detach().cpu()
                soft_tense_logit = softmax_layer(tense_logit).detach().cpu()
                soft_certainty_logit = softmax_layer(certainty_logit).detach().cpu()

                if i==0:
                    type_preds += soft_type_logit
                    polarity_preds += soft_polarity_logit
                    tense_preds += soft_tense_logit
                    certainty_preds += soft_certainty_logit
                else:
                    tmp_type_preds += soft_type_logit
                    tmp_polarity_preds += soft_polarity_logit
                    tmp_tense_preds += soft_tense_logit
                    tmp_certainty_preds += soft_certainty_logit

        if i !=0:
            for j in range(len(type_preds)):
                type_preds[j] += tmp_type_preds[j]
                polarity_preds[j] += tmp_polarity_preds[j]
                tense_preds[j] += tmp_tense_preds[j]
                certainty_preds[j] += tmp_certainty_preds[j]
    
    for k in range(len(type_preds)):
        type_preds[k] = int(type_preds[k].argmax())
        polarity_preds[k] = int(polarity_preds[k].argmax())
        tense_preds[k] = int(tense_preds[k].argmax())
        certainty_preds[k] = int(certainty_preds[k].argmax())

    print("inference ensemble finished")
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@")

                    
    return type_preds, polarity_preds, tense_preds, certainty_preds


In [4]:
type_preds, polarity_preds, tense_preds, certainty_preds = inference_ensemble(os.path.join(args.saved_path,args.model_name), test_loader, device)

type_le = label_encoder['type']
polarity_le = label_encoder['polarity']
tense_le = label_encoder['tense']
certainty_le = label_encoder['certainty']

type_preds = type_le.inverse_transform(type_preds)
polarity_preds = polarity_le.inverse_transform(polarity_preds)
tense_preds = tense_le.inverse_transform(tense_preds)
certainty_preds = certainty_le.inverse_transform(certainty_preds)

predictions = []
for type_pred, polarity_pred, tense_pred, certainty_pred in zip(type_preds, polarity_preds, tense_preds, certainty_preds):
    predictions.append(type_pred+'-'+polarity_pred+'-'+tense_pred+'-'+certainty_pred)

submit = pd.read_csv(os.path.join(args.data_path,'sample_submission.csv'))
submit['label'] = predictions

@@@@@@@@@@@@@@@@@@@@@
['k0_roberta_document_sds', 'k1_roberta_document_sds', 'k2_roberta_document_sds', 'k3_roberta_document_sds', 'k4_roberta_document_sds']
@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@
model_load start
model : model_0_mean_f1.pth
model_state_dict : model(best_scores)_0_mean_f1.pth
model loaded
@@@@@@@@@@@@@@@@


  0%|          | 0/111 [00:00<?, ?it/s]

@@@@@@@@@@@@@@@@
model_load start
model : model_1_mean_f1.pth
model_state_dict : model(best_scores)_1_mean_f1.pth
model loaded
@@@@@@@@@@@@@@@@


  0%|          | 0/111 [00:00<?, ?it/s]

@@@@@@@@@@@@@@@@
model_load start
model : model_2_mean_f1.pth
model_state_dict : model(best_scores)_2_mean_f1.pth
model loaded
@@@@@@@@@@@@@@@@


...

@@@@@@@@@@@@@@@@
model_load start
model : model_3_mean_f1.pth
model_state_dict : model(best_scores)_3_mean_f1.pth
model loaded
@@@@@@@@@@@@@@@@


  0%|          | 0/111 [00:00<?, ?it/s]

@@@@@@@@@@@@@@@@
model_load start
model : model_4_mean_f1.pth
model_state_dict : model(best_scores)_4_mean_f1.pth
model loaded
@@@@@@@@@@@@@@@@


  0%|          | 0/111 [00:00<?, ?it/s]

inference ensemble finished
@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [5]:
now_date = datetime.now()
diff_hours = timedelta(hours=9)
now_date += diff_hours
print_now = str(now_date.month) + '_' + str(now_date.day) + '_' + str(now_date.hour) + '_' + str(now_date.minute)

if not os.path.exists(args.output_path):
    os.makedirs(args.output_path)

if not os.path.exists(os.path.join(args.output_path,'soft_ensemble')):
    os.makedirs(os.path.join(args.output_path,'soft_ensemble'))

submit.to_csv(os.path.join(args.output_path,'soft_ensemble',f'{args.model_name}_{print_now}.csv'), index = False)

print("Inference Finish")

Inference Finish


# 비교

In [6]:
bigo = pd.read_csv('./results/hard_ensemble/hard_ensemble_final_submission_12_21_15_44.csv')
submit['bigyo'] = bigo['label']
submit[submit['label']!=submit['bigyo']]

,ID,label,bigyo
3,TEST_0003,사실형-긍정-과거-확실,사실형-긍정-현재-확실
8,TEST_0008,사실형-긍정-현재-확실,사실형-긍정-미래-확실
12,TEST_0012,사실형-긍정-과거-확실,추론형-긍정-과거-확실
18,TEST_0018,사실형-긍정-과거-확실,추론형-긍정-과거-확실
42,TEST_0042,추론형-긍정-미래-불확실,추론형-긍정-미래-확실
...,...,...,...
7020,TEST_7020,사실형-긍정-미래-불확실,사실형-긍정-미래-확실
7032,TEST_7032,추론형-긍정-현재-확실,사실형-긍정-현재-확실
7077,TEST_7077,대화형-미정-과거-불확실,대화형-긍정-과거-불확실
7083,TEST_7083,추론형-긍정-현재-불확실,추론형-긍정-현재-확실
